In [ ]:
#libraries
library(pracma)
library(ggplot2)
library(maps) 
#user functions
source('R_lib/regression/inla/regression_inla_model1_unbounded_hyp.R')

In [ ]:
#regression name
reg_name  <- 'SD_regression_ds1_inla'

#regression dataset
flatfile_fname <- 'data/new_SD_PGA.csv'

#output directory
dir_out <- 'data/SD_ds1_inla/'

#INLA regression parameters
runinla_flag <- TRUE # TRUE or FALSE
res_name     <- 'PGA'

#mesh coarseness
# #fine
# mesh_edge_max     <- 5
# mesh_inner_offset <- 15
# mesh_outer_offset <- 15
# #medium
# mesh_edge_max     <- 15
# mesh_inner_offset <- 15
# mesh_outer_offset <- 50
#coarse
mesh_edge_max     <- 50
mesh_inner_offset <- 50
mesh_outer_offset <- 150

#approximation options
# if flag_gp_approx=TRUE uses int.strategy="eb" and strategy="gaussian"
# int.strategy="eb" corresponds to one integration point, and 
# strategy="gaussian" approximates posteriors as gaussian distributions
flag_gp_approx <- TRUE # TRUE or FALSE

#number of threads
# reduce number of threads if running out of memmory, if not specified
# number of CPU threads is used
n_threads <- 8

In [ ]:
#load flatfile
utmzone = '10n'
df_flatfile <- read.csv(flatfile_fname)
names(df_flatfile) <- c('eqid','date','eqlat','eqlon','dep','mag','SD','SDD','Site','R','Vs30','Vs30class','PGA','stalat','stalon')
df_flatfile$ssn <- df_flatfile$Site
df_flatfile$UTMzone <- utmzone #north california

eqx<-LongLatToUTM(df_flatfile$eqlat,df_flatfile$eqlon,utmzone)
df_flatfile$eqX <- eqx[,'X']
df_flatfile$eqY <- eqx[,'Y']

stax<-LongLatToUTM(df_flatfile$stalat,df_flatfile$stalon,utmzone)
df_flatfile$staX <- stax[,'X']
df_flatfile$staY <- stax[,'Y']

df_flatfile

In [ ]:
#run INLA model
out_fname <- reg_name



# Preprocess Input Data
# ---------------------------
n_data <- nrow(df_flatfile)
#earthquake data
data_eq_all <- df_flatfile[,c('eqid','mag','eqX', 'eqY')]
out_unq  <- UniqueIdxInv(df_flatfile[,'eqid'])
eq_idx   <- out_unq$idx
eq_inv   <- out_unq$inv
data_eq  <- data_eq_all[eq_idx,]
X_eq     <- data_eq[,c(3,4)] #earthquake coordinates
X_eq_all <- data_eq_all[,c(3,4)]
#create earthquake ids for all records (1 to n_eq)
eq_id <- eq_inv
n_eq  <- nrow(data_eq)
  


In [ ]:
#station data
data_sta_all <- df_flatfile[,c('ssn','Vs30','staX','staY')]
out_unq   <- UniqueIdxInv(df_flatfile[,'ssn'])
sta_idx   <- out_unq$idx
sta_inv   <- out_unq$inv
data_sta  <- data_sta_all[sta_idx,]
X_sta     <- data_sta[,c(3,4)] #station coordinates
X_sta_all <- data_sta_all[,c(3,4)]
#create station indices for all records (1 to n_sta)
sta_id <- sta_inv
n_sta  <- nrow(data_sta)

In [ ]:
#ground-motion observations  
y_data <- df_flatfile[,res_name]

In [ ]:
#UTM zone
utm_zone <- unique(df_flatfile$UTMzone)
utm_no   <- as.numeric(gsub("([0-9]+).*$", "\\1", utm_zone))

In [ ]:
# Run INLA, fit model 
# ---------------------------
#fixed effects 
#---   ---   ---   ---   ---   ---
#prior on the fixed effects
prior_fixed <- list(mean.intercept = 0, prec.intercept = 5,
                    mean = (list(intcp=0.0, default=0)),
                    prec = (list(intcp=5.0, default=0.01)))

#covariates
df_inla_covar <- data.frame(intcp = 1, eq = eq_id, sta = sta_id)

In [ ]:
#spatial model
#---   ---   ---   ---   ---   ---
#input arguments
edge_max     <- mesh_edge_max
inner_offset <- mesh_inner_offset
outer_offset <- mesh_outer_offset

#domain mesh
mesh <- inla.mesh.2d(loc=rbind(as.matrix(X_eq),as.matrix(X_sta)) ,
                    max.edge = c(1,5)*edge_max,
                    cutoff = 3, offset = c(inner_offset, outer_offset))